# 1. GPT 모델

기존의 encoder-decoder 구조인 Transformer 모델을 변형하여 GPT 모델 구조를 만들기 위해서는 아래와 같은 변경이 필요합니다. 이때 빨간색 박스는 사라져야할 컴포넌트이며, 초록색 박스는 변경되어야 하는 컴포넌트를 의미합니다.
![](https://github.com/minkj1992/ai/blob/main/static/Untitled-2024-06-21-1948.png?raw=true)

openai에서 발표한 [Improving Language Understanding by Generative Pre-Training](https://s3-us-west-2.amazonaws.com/openai-assets/research-covers/language-unsupervised/language_understanding_paper.pdf) 논문에 따르면, decoder 모델을 토대로 generalized한 pre-trained 모델을 만듭니다. 그러므로 transformer의 사라져야할 부분은 아래와 같습니다.

#### 사라져야할 부분
> 빨간 네모
1. `Encoder`
2. `encoder-decoder attention layer`


위 논문에 따르면 변경되어야할 부분은 아래와 같습니다.

#### 변경되어야 할 부분
> 초록 네모

1. Positional encoding -> Positional embedding
2. Target input -> `Task-specific input transformations`

해당 논문에서는 또한 기존의 sin,cos을 활용한 positional 계산 방식을 positional embedding (max_sentence_len, d_model)을 생성하여 학습되도록 합니다. 

Target input 또한 달라져야 하는데, 이번 시간에는 pretrain + fine tunning 총 2번 학습했던 논문과 달리 1번만 학습할 예정이기 때문에, input을 변형해서 넣어주도록 하였습니다. 

현재 데이터 셋은 사용자의 질문인, df['Q']와 챗봇의 답변인 df['A']가 존재하며, 이 두 데이터 모두 챗봇에 필요하며 부족한 데이터를 고려하여, 합쳐서 학습해주도록 하겠습니다.


In [44]:
from tqdm.notebook import tqdm
tqdm.pandas()
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [45]:
path = os.getenv('HOME')+'/aiffel/transformer_chatbot/data/ChatbotData .csv' 
origin = pd.read_csv(path) 

print(origin.shape)
origin.sample(15)

(11823, 3)


,Q,A,label
10819,오랫동안 썸만 타. 뭐가 문제임?,고백을 안 한 게 문제일 거 같아요.,2
4659,친구랑 싸웠어,싸우면서 정 들 거예요.,0
6939,어떻게 살아가야 할까,태연하게 시작해보세요.,1
376,기분이 이상해,무슨 이유인지 생각해보세요.,0
181,고기 먹고 싶어,저기압에는 고기앞이죠.,0
3058,어이없어,그냥 잊어버리세요.,0
1891,바다 가자고 하면 갈까?,같이 가자고 말해보세요,0
7213,온전히 내 편이길 바라는 누군가가 옆에 있었으면 좋겠는,제가 당신 곁에 있어드릴게요.,1
5893,내여잘 건드린놈들,벌 받을 거예요.,1
9591,다시 웃게 해준 사람,그 사람이 당신의 홈닥터군요.,2


# 1. Preprocess

In [48]:
tmp = origin.copy()

df_q = pd.DataFrame({
    'text': tmp['Q']
})
df_a = pd.DataFrame({
    'text': tmp['A']
})

df = pd.concat([df_q, df_a], ignore_index=True)

print(df.shape)
df.sample(15)

(23646, 1)


,text
5448,감정조절이 안되서 문자 보냈네
10735,연애보다 썸이 좋아.
13404,서로 마음만 맞으면 가능해요.
12714,공부한 만큼 나올 거예요.
2658,스키 강습 받아야 될까?
12926,꿈같은 이야기네요.
2763,식욕이 없어
5973,너한테 쓰는 편지
8412,하루하루가 지옥같아
6758,슬픈 예감대로 되어가는 현실


In [49]:
def preprocess(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^가-힣a-zA-Z?.!,]+", " ", sentence)
    sentence = sentence.strip()
    return sentence

def postprocess(df):
    df.replace('', np.nan, inplace=True)
    df.dropna(subset=['text'], inplace=True)

print(df.shape)
df.drop_duplicates(subset = ['text'], inplace = True)
df['text'] = df['text'].progress_apply(preprocess)
postprocess(df)
print(df.shape)

(23646, 1)


  0%|          | 0/19436 [00:00<?, ?it/s]

(19436, 1)


## 2. Data prepare

- 질문,답변 각자 적절한 maximum 크기만 남기고 지운다.
- `[SOS] 질문 [DELIM] 답변 [EOS]` 합친다 -> teacher forcing 학습
- 추론단계에서는 사용자 질문 -> 챗봇 답변에서 [DELIM] ~ [EOS]까지만 return.


## (WIP) 추후 도입 방법
1. Genralized Pretrain
2. Fine tunning

두가지를 수행하기 위해서 2가지에 필요한 데이터를 준비하겠습니다.

1번을 위해서는 하나의 row에 속한 df['Q']와 df['A']를 2개의 row로 나눠서 df['text']로 만들어 0...i -> i+1예측 teacher forcing을 합니다.

이후 2번은 `[SOS] Q [DELIM] A_set [EOS]`형태의 문장을 A_set만큼 만들어, 정답인 A를 예측하도록 하는 softmax를 생성, 이를 위해서는 정답이 아닌 negative sampling이 필요해서, 다음 기회에 시도


In [52]:
def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if len(s.split()) <= max_len:
            cnt = cnt + 1
    print(
        "전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s"
        % (max_len, (cnt / len(nested_list)))
    )

    
len_text = [len(s.split()) for s in df['text']]
max_len_text = 9
print("text의 최소 길이 : {}".format(np.min(len_text)))
print("text의 최대 길이 : {}".format(np.max(len_text)))
print("text의 평균 길이 : {}".format(np.mean(len_text)))
below_threshold_len(max_len_text, df["text"])

text의 최소 길이 : 1
text의 최대 길이 : 24
text의 평균 길이 : 4.315651368594361
전체 샘플 중 길이가 9 이하인 샘플의 비율: 0.9842045688413253


In [53]:
before = len(df)
def is_within(text, max_len):
    return len(text.split()) <= max_len
_filter = df["text"].apply(is_within, max_len=max_len_text)
df = df[_filter]
print("전체 샘플수 :", (len(df)))
print(f"삭제된 샘플수: {before - len(df)}")

전체 샘플수 : 19129
삭제된 샘플수: 307


In [54]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    df["text"],
    target_vocab_size=2**13,
)
SOS, EOS = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
print("SOS 번호 :", [tokenizer.vocab_size])
print("EOS 번호 :", [tokenizer.vocab_size + 1])
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

SOS 번호 : [8817]
EOS 번호 : [8818]
8819


In [68]:
def tokenize(texts):
    tokenized = []
    max_len = 0
    for text in texts:
        tokenized_txt = SOS + tokenizer.encode(text) + EOS
        max_len = max(max_len, len(tokenized_txt))
        tokenized.append(tokenized_txt)

    # max_length 으로 모든 데이터셋을 패딩
    return tf.keras.preprocessing.sequence.pad_sequences(
        tokenized, maxlen=max_len, padding="post"
    ), max_len


texts, MAX_LENGTH = tokenize(df["text"])
print(MAX_LENGTH, len(texts))

20 19129


In [78]:
texts[0].shape
len(texts)

19129

In [70]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': texts[:,:-1],
    },
    {
        'outputs': texts[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

# 모델 학습

In [71]:
def scaled_dot_product_attention(query, key, value, mask):
    """
    query: (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    key: (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
    value: (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
    padding_mask : (batch_size, 1, 1, key의 문장 길이)
    """

    matmul_qk = tf.matmul(query, key, transpose_b=True)

    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    if mask is not None:
        logits += mask * -1e9  # FYI, 0은 softmax에서 양수값을 가진다.

    attention_weights = tf.nn.softmax(logits, axis=-1)

    # output : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    output = tf.matmul(attention_weights, value)
    return output, attention_weights


class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        # d_model을 num_heads로 나눈 값.
        # 논문 기준 : 64 (512 // 8)
        self.depth = d_model // self.num_heads

        # WQ, WK, WV에 해당하는 밀집층 정의
        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        # WO에 해당하는 밀집층 정의
        self.dense = tf.keras.layers.Dense(units=d_model)

    # num_heads 개수만큼 q, k, v를 split하는 함수
    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(
            inputs, perm=[0, 2, 1, 3]
        )  # (batch, heads, max 문장 토큰 갯수, 64)

    def call(self, inputs):
        query, key, value, mask = (
            inputs["query"],
            inputs["key"],
            inputs["value"],
            inputs["mask"],
        )
        batch_size = tf.shape(query)[0]

        # 1. WQ, WK, WV에 해당하는 밀집층 지나기
        # q : (batch_size, query의 문장 길이, d_model)
        # k : (batch_size, key의 문장 길이, d_model)
        # v : (batch_size, value의 문장 길이, d_model)
        # 참고) 인코더(k, v)-디코더(q) 어텐션에서는 query 길이와 key, value의 길이는 다를 수 있다.
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # 2. 헤드 나누기
        # q : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
        # k : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
        # v : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
        # (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
        scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
        # (batch_size, query의 문장 길이, num_heads, d_model/num_heads)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # 4. 헤드 연결(concatenate)하기
        # (batch_size, query의 문장 길이, d_model)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

        # 5. WO에 해당하는 밀집층 지나기
        # (batch_size, query의 문장 길이, d_model)
        outputs = self.dense(concat_attention)

        return outputs

In [128]:
def create_padding_mask(x):
    # x (batch_size, max 문장 토큰 수)
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]


# 가릴곳: 1, 참조할곳: 0
def create_look_ahead_mask(x):

    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

# https://github.com/tensorflow/models/blob/952b8f05f42aaf27b083d0cd53946a2a2e9a4c69/official/nlp/modeling/layers/position_embedding.py#L27
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(
        self,
        sequence_length,
        initializer="he_normal",
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.sequence_length = int(sequence_length)
        self.initializer = tf.keras.initializers.get(initializer)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "initializer": tf.keras.initializers.serialize(self.initializer),
            }
        )
        return config

    def build(self, inputs_shape):
        feature_size = inputs_shape[-1]
        self.position_embeddings = self.add_weight(
            name="embeddings",
            shape=[self.sequence_length, feature_size],
            initializer=self.initializer,
            trainable=True,
        )
        self.built = True

    def call(self, inputs, start_index=0):
        shape = tf.shape(inputs)
        feature_length = shape[-1]
        sequence_length = shape[-2]
        position_embeddings = tf.convert_to_tensor(self.position_embeddings)
        position_embeddings = tf.slice(
            position_embeddings,
            (start_index, 0),
            (sequence_length, feature_length),
        )
        return tf.broadcast_to(position_embeddings, shape)

    def compute_output_shape(self, input_shape):
        return input_shape    
    
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")

    # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
    attention1 = MultiHeadAttention(d_model, num_heads, name="attention_1")(
        inputs={
            "query": inputs,
            "key": inputs,
            "value": inputs,
            "mask": look_ahead_mask,
        }
    )
    attention1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention1 + inputs)


    # 두 번째 서브 레이어 : 2개의 완전연결층
    outputs = tf.keras.layers.Dense(units=units, activation="relu")(attention1)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs + attention1)
    return tf.keras.Model(
        inputs=[inputs, look_ahead_mask],
        outputs=outputs,
        name=name,
    )


def decoder(vocab_size, num_layers, units, d_model, num_heads, dropout, seq_length, name="decoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")

    # 임베딩 레이어
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

    # 포지셔널 임베딩
    embeddings = PositionalEmbedding(seq_length)(embeddings)
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="decoder_layer_{}".format(i),
        )(inputs=[outputs, look_ahead_mask])

    return tf.keras.Model(
        inputs=[inputs, look_ahead_mask],
        outputs=outputs,
        name=name,
    )


In [129]:
def transformer(
    vocab_size, num_layers, units, d_model, num_heads, dropout, seq_length, name="transformer",
):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    look_ahead_mask = tf.keras.layers.Lambda(
        create_look_ahead_mask, output_shape=(1, None, None), name="look_ahead_mask"
    )(inputs)

    # 디코더
    outputs = decoder(
        vocab_size=vocab_size,
        num_layers=num_layers,
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        seq_length=seq_length,
    )(inputs=[inputs, look_ahead_mask])

    # 완전연결층
    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(outputs)

    return tf.keras.Model(
        inputs=[
            inputs,
        ],
        outputs=outputs,
        name=name,
    )


In [130]:
tf.keras.backend.clear_session()

def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction="none"
    )(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)


# 하이퍼파라미터
NUM_LAYERS = 12  # GPT의 경우 인코더 층의 개수로 설정됨
D_MODEL = 768  # GPT 모델에서 사용된 모델 크기
NUM_HEADS = 12  # GPT에서 사용된 멀티 헤드 어텐션의 헤드 수
UNITS = 3072  # GPT에서 사용된 피드 포워드 신경망의 은닉층 크기
DROPOUT = 0.1  # GPT에서 사용된 드롭아웃 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT,
    seq_length=MAX_LENGTH,
)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
look_ahead_mask (Lambda)        (None, 1, None, None 0           inputs[0][0]                     
__________________________________________________________________________________________________
decoder (Functional)            (None, None, 512)    12936704    inputs[0][0]                     
                                                                 look_ahead_mask[0][0]            
__________________________________________________________________________________________________
outputs (Dense)                 (None, None, 8819)   4524147     decoder[0][0]          

In [131]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


learning_rate = CustomSchedule(D_MODEL)
optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9
)
def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


model.compile(
    optimizer=optimizer, 
    loss=loss_function, 
    metrics=[accuracy],
)

history = model.fit(dataset, epochs=20, verbose=1)

Epoch 1/20


299/299 [==============================] - 19s 52ms/step - loss: 2.6307 - accuracy: 0.0469
Epoch 2/20
299/299 [==============================] - 15s 51ms/step - loss: 2.2900 - accuracy: 0.0544
Epoch 3/20
299/299 [==============================] - 15s 50ms/step - loss: 2.2587 - accuracy: 0.0544
Epoch 4/20
299/299 [==============================] - 15s 50ms/step - loss: 2.2496 - accuracy: 0.0544
Epoch 5/20
299/299 [==============================] - 15s 50ms/step - loss: 2.2454 - accuracy: 0.0544
Epoch 6/20
299/299 [==============================] - 15s 50ms/step - loss: 2.2418 - accuracy: 0.0544
Epoch 7/20
299/299 [==============================] - 15s 50ms/step - loss: 2.2399 - accuracy: 0.0544
Epoch 8/20
299/299 [==============================] - 15s 50ms/step - loss: 2.2365 - accuracy: 0.0544
Epoch 9/20
299/299 [==============================] - 15s 50ms/step - loss: 2.2337 - accuracy: 0.0544
Epoch 10/20
299/299 [==============================] - 15s 50ms/step - loss: 2.2354 - accurac

In [132]:
def decoder_inference(sentence):
    sentence = preprocess(sentence)
    sentence = tf.expand_dims(
        SOS + tokenizer.encode(sentence), axis=0
    )
    output_sequence = tf.expand_dims(SOS, 0)
    for i in range(MAX_LENGTH):
        predictions = model(inputs=[sentence], training=False)
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if tf.equal(predicted_id, EOS[0]):
            break
        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)
    return tf.squeeze(output_sequence, axis=0)


def sentence_generation(sentence):    
    prediction = decoder_inference(sentence)
    predicted_sentence = tokenizer.decode(
        [i for i in prediction if i < tokenizer.vocab_size]
    )
    print(f"🧑 : {sentence}")
    print(f"🤖 : {predicted_sentence}")

In [134]:
sentence_generation('안녕하세요!')

🧑 : 안녕하세요!
🤖 : 


In [135]:
sentence_generation('실패!!!!')

🧑 : 실패!!!!
🤖 : 
